In [1]:
import pandas as pd

csvname = '/home/anton/Documents/Tulane/Research/Work/UKB/Ethnicities.csv'
df = pd.read_csv(csvname)
df

,Participant ID,Ethnic background | Instance 0,Ethnic background | Instance 1,Ethnic background | Instance 2
0,1000260,"[""Any other Asian background""]",NaN,NaN
1,1000693,"[""Indian""]",NaN,NaN
2,1001366,"[""Pakistani""]",NaN,NaN
3,1001824,"[""Caribbean""]",NaN,NaN
4,1002126,"[""African""]",NaN,NaN
...,...,...,...,...
19505,6024015,"[""Caribbean""]",NaN,NaN
19506,6024023,"[""Indian""]",NaN,NaN
19507,6024230,"[""African""]",NaN,NaN
19508,6024409,"[""Caribbean""]",NaN,NaN


In [13]:
import pickle
import re

demodir = '/home/anton/Documents/Tulane/Research/ImageNomer/data/anton/cohorts/UKB/'

demo = pickle.load(open(f'{demodir}/demographics.pkl', 'rb'))
race_counts = dict(Caucasian=0)

demo['f_21000_0_0'] = dict()
demo['f_21000_0_0_comb'] = dict() 

for sub in demo['f_21003_2_0']:
    res = df[df['Participant ID'] == sub]
    if not res.empty:
        bg = res['Ethnic background | Instance 0'].item()
        match = re.match('\["([a-zA-z ]+)"\]', bg)
        if not match:
            print(bg)
            break
        r = match.group(1)
        if r == 'Any other Asian background':
            r = 'Other Asian'
        if r == 'Any other Black background':
            r = 'African'
        if r == 'Asian or Asian British':
            r = 'Other Asian'
        if r in race_counts:
            race_counts[r] += 1
        else:
            race_counts[r] = 1
        if r in ['African', 'Caribbean']:
            demo['f_21000_0_0_comb'][sub] = 'African'
        demo['f_21000_0_0'][sub] = r
    else:
        race_counts['Caucasian'] += 1
        demo['f_21000_0_0'][sub] = 'Caucasian'
        demo['f_21000_0_0_comb'][sub] = 'Caucasian'
            
print(race_counts)
print(len(demo['f_21000_0_0']))

{'Caucasian': 39811, 'Chinese': 115, 'African': 115, 'Indian': 289, 'Caribbean': 153, 'Other Asian': 69, 'Pakistani': 65, 'Bangladeshi': 6}
40623


In [14]:
# Write race data

pickle.dump(demo, open(f'{demodir}/demographics.pkl', 'wb'))

print('Complete')

Complete


In [53]:
import pickle

field = 25753
scan = 2

basedir = '/run/media/anton/AntonData1/UKB_MRI_data_11092022/'

fc = pickle.load(open(f'{basedir}/{field}_{scan}.pkl', 'rb'))

print(len(fc))

40471


In [54]:
# Pack data for race prediction

import numpy as np

x0 = []
x1 = []

for sub in fc:
    try:
        if demo['f_22001_0_0'][sub] == 1:
            x1.append(fc[sub])
        elif demo['f_22001_0_0'][sub] == 0:
            x0.append(fc[sub])
    except:
        pass
    
x0 = np.stack(x0)
x1 = np.stack(x1)

print(len(x0), len(x1))

20500 18266


In [55]:
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split

errs = []
nullerrs = []

for _ in range(10):
    idcs = np.random.permutation(len(x0))
    x0m = x0[idcs[:300]]
    idcs = np.random.permutation(len(x1))
    x1m = x1[idcs[:300]]
    race = np.concatenate([np.zeros(len(x0m)), np.ones(len(x1m))])
    
    x = np.concatenate([x0m, x1m])
    xtr, xt, ytr, yt = train_test_split(x, race, stratify=race, train_size=0.8)

    reg = LogisticRegression(max_iter=1000).fit(xtr, ytr)
    yhat = reg.predict(xt)
    err = np.mean(yhat == yt)
    nullerr = np.mean(yt)
    if nullerr < 0.5:
        nullerr = 1-nullerr
    print(err)
    print(nullerr)
    errs.append(err)
    nullerrs.append(nullerr)
    
print(np.mean(errs), np.std(errs))

0.8416666666666667
0.5
0.875
0.5
0.8416666666666667
0.5
0.8833333333333333
0.5
0.8416666666666667
0.5
0.8333333333333334
0.5
0.85
0.5
0.8916666666666667
0.5
0.8833333333333333
0.5
0.825
0.5
0.8566666666666667 0.022912878474779203
